In [ ]:
import os
import rasterio
from rasterio import windows
from rasterio.enums import Resampling
import numpy as np

from concurrent.futures import ProcessPoolExecutor

In [ ]:
file_list = r"/media/ross/ssd/00_2015_DAR_marinedebris/kahoolawe/03_gdalwarp_blocksize/tif_list.txt"
out_dir = r"/media/ross/ssd/00_2015_DAR_marinedebris/kahoolawe/04_window_retile/"

In [ ]:
def windowed_retile(in_path):
    with rasterio.open(in_path, 'r') as src:
        src_transform = src.transform
        src_crs = src.crs

        assert len(set(src.block_shapes)) == 1

        for ji, current_window in src.block_windows():
            print(ji)
            #get the affine matrix for the window we are writing
            win_transform = src.window_transform(current_window)
            
            #Play with the file names, end up with our output path
            basename = os.path.basename(in_path)
            basename_no_ext = os.path.splitext(basename)[0]
            filename = f"{basename_no_ext}_{ji[0]}_{ji[1]}.jpg"
            out_path = os.path.join(out_dir, filename)
            
            
            #get the actual data from the window as a 3D numpy array
            data = src.read(window=current_window)
            #check if our data is all zeros (no data), if it is don't write the file
            zeros = np.zeros_like(data)

            if np.array_equal(data, zeros):
                print(f"{filename} is no data")

            else:
                #this only runs if we have valid data. Write a custom JPEG profile, and export our window to a file.
                profile={'driver': 'JPEG',
                        'count': src.count,
                        'dtype': rasterio.ubyte,
                        'height': current_window.height,
                        'width': current_window.width,
                        'transform': win_transform,
                        'crs': src_crs}

                with rasterio.open(out_path, 'w', **profile) as dst:
                    dst.write(data)

                print(f"wrote {out_path}")

In [ ]:
with open(file_list, 'r') as f:
    in_paths = [line.strip() for line in f]

num_of_files = len(in_paths)
print(f"{num_of_files} total images to process")

for path in in_paths:
    windowed_retile(path)

'''
with ProcessPoolExecutor() as executor:
    executor.map(windowed_retile, in_paths)
'''